## Convert MU69 halo surface brightness into particle density. 

In [42]:
def fac2mag(fac):
    import numpy as np
    return np.log( 1/(1.*fac))/np.log(100**0.2)    

def mag2fac(mag): 
    import numpy as np
    return np.exp(-mag * np.log(100.**0.2))

In theory I could do this theoretically. There should be one published correct value for (mag per arcsec2, at a given distance) to (surface albedo). Assuming zero phase.

I mean, mags per arcsec2 -- that's should translate directly into an amount of flux.

Maybe just bootstrap off of the sun: -26.7 = 1400 W/m2. That's probably the best.

In [56]:
import numpy as np
import astropy
import astropy.units as u
import astropy.constants as c
import math

In [85]:
mag_sun  = -26.7
dist_kbo = 43.4*u.AU  # Not sure of the current distance. Mostly circular.: 42 .. 46 AU.
pi       = math.pi    # Just for convenience
dx       = dist_kbo * 1*u.arcsecond.to('radian') # Calc the dim's of a 1 arcsec 
                                                 # paper target as seen from Sun

Let's put a 1 arcsec piece of white paper at 43 AU. How much solar flux does it reflect back to Earth?

In [51]:
L_paper = c.L_sun * 1/(4*pi*(dist_kbo)**2) * dx**2 * 1/(4*pi*(dist_kbo)**2)

How big is that paper?

In [52]:
dx.to('km')

<Quantity 31476.75896101348 km>

In [57]:
L_paper.to('W/m**2')

<Quantity 1.3580288693349867e-12 W / m2>

Now, how many magnitudes per arcsec is this? Sun has a total brightness of -26.7, as seen on Earth. How many times fainter is this white piece of paper, than the Sun would be, at Earth?

In [54]:
ratio = L_paper / (c.L_sun / (4*pi*(1*u.AU)**2))

And now print the magnitude of this paper. Since it is exactly 1" across, this **is** the number of mags per arcsec already.

In [59]:
mag_paper  = mag_sun + fac2mag(ratio)  # Mag of white paper, seen from Earth
mpas_paper = mag_paper
print("Mags/arcsec of white paper at 43 AU = {}".format(mpas_paper))

Mags/arcsec of white paper at 43 AU = 10.807598973500095


Now, we know the mags per arcsec of the region around MU69, as measured by Alex Parker. That means we can just calculate the 'filling factor' -- that is, how much fainter is that region, than a 1" piece of paper.

In [100]:
mpas_kbo_hst = 28            # Mags per arcsec. Alex Parker limit
albedo_kbo   = 0.2           # Albedo assumed for the dust. 
r_dust       = 50*u.micron   # Size of the dust. A bit smaller than the sc safety limit.
r_dust       = 0.2*u.mm      # Size of the dust, in my whitepaper
A_sc         = 5*(u.meter)**2 # S/C cross-section, in my whitepaper

In [113]:
fillfac = mag2fac(mpas_kbo_hst - mpas_paper)
print ("Geometric filling factor = {}".format(fillfac))

Geometric filling factor = 1.3275154658191067e-07


Now convert to particles per square meter, and bring in albedo.

In [102]:
n = fillfac * ((1*u.meter**2) / (pi * r_dust**2)).to('1') / albedo_kbo

In [112]:
print("{} particles of radius {} per square meter".format(n, r_dust))
print("{} particles hitting s/c during encounter".format(n*A_sc.to('m2').value))

5.282016210401271 particles of radius 0.2 mm per square meter
26.410081052006355 particles hitting s/c during encounter


### Validate that my solar flux is correct

In [95]:
(c.L_sun / (4*pi*(1*u.AU)**2)).to('W/m2')

<Quantity 1367.5669346816567 W / m2>

In [109]:
A_sc.to('m2').value

5.0